<a href="https://colab.research.google.com/github/Jaksta1/Monte-Carlo-method-for-option-prices/blob/main/skrypt%20mgr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from scipy.stats import norm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from qmcpy import Sobol

# ---------------------------
# QMC -> normals (QMCPy Sobol, randomize=True gives full Owen scramble)
# ---------------------------
def qmc_normals_qmcpy(n_paths, total_dim, seed=None):
    """Zwraca tablicę (n_paths, total_dim) z punktami Sobola przekształconymi do N(0,1)."""
    sob = Sobol(dimension=total_dim, randomize=True, seed=seed)
    u = sob.gen_samples(n_paths)  # shape (n_paths, total_dim)
    eps = 1e-16
    u = np.clip(u, eps, 1 - eps)
    return norm.ppf(u)


# ---------------------------
# Rozszerzony MARS (LDA + GCV)
# ---------------------------
class ExtendedMARS:
    def __init__(self, max_terms=20, min_leaf=10, penalty=3.0):
        self.max_terms = int(max_terms)
        self.min_leaf = int(min_leaf)
        self.penalty = float(penalty)
        self.basis = []   # list of (a, knot, sign)
        self.coefs = None
        self.intercept_ = 0.0

    def _project(self, X, a):
        return X.dot(a)

    def _gcv(self, y, y_pred, n_params):
        n = len(y)
        rss = np.sum((y - y_pred) ** 2)
        C = n_params + self.penalty
        if n <= C:
            return rss  # fallback
        denom = (1.0 - C / n) ** 2
        return rss / denom

    def fit(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y).ravel()
        n, d = X.shape
        design = np.ones((n, 1))  # intercept
        self.basis = []

        for step in range(self.max_terms):
            coef_curr, *_ = np.linalg.lstsq(design, y, rcond=None)
            y_pred_curr = design.dot(coef_curr)
            residual = y - y_pred_curr

            # candidate directions: LDA on sign of residuals (two classes)
            try:
                labels = (residual > np.median(residual)).astype(int)
                lda = LinearDiscriminantAnalysis(n_components=1)
                lda.fit(X, labels)
                a_candidate = lda.coef_[0]
                if np.linalg.norm(a_candidate) > 0:
                    a_candidate = a_candidate / (np.linalg.norm(a_candidate) + 1e-12)
                    candidate_dirs = [a_candidate]
                else:
                    candidate_dirs = [np.eye(d)[j] for j in range(d)]
            except Exception:
                candidate_dirs = [np.eye(d)[j] for j in range(d)]

            best_gcv = np.inf
            best_tuple = None  # (a, knot, sign, col)

            for a in candidate_dirs:
                proj = self._project(X, a)
                # candidate knots: percentyle w zakresie [10,90]
                nknots = max(2, min(12, n // max(1, self.min_leaf)))
                percentiles = np.linspace(10, 90, nknots)
                knots = np.unique(np.percentile(proj, percentiles))
                for knot in knots:
                    for sign in (+1, -1):
                        if sign == 1:
                            col = np.maximum(proj - knot, 0.0)[:, None]
                        else:
                            col = np.maximum(knot - proj, 0.0)[:, None]
                        if np.allclose(col, 0.0):
                            continue
                        D = np.hstack([design, col])
                        coef_try, *_ = np.linalg.lstsq(D, y, rcond=None)
                        y_try = D.dot(coef_try)
                        gcv = self._gcv(y, y_try, D.shape[1])
                        if gcv < best_gcv - 1e-12:
                            best_gcv = gcv
                            best_tuple = (a.copy(), float(knot), int(sign), col)

            if best_tuple is None:
                break
            # accept basis
            a_best, knot_best, sign_best, col_best = best_tuple
            self.basis.append((a_best, knot_best, sign_best))
            design = np.hstack([design, col_best])

        # final fit
        coef_final, *_ = np.linalg.lstsq(design, y, rcond=None)
        self.intercept_ = float(coef_final[0])
        if design.shape[1] > 1:
            self.coefs = coef_final[1:].astype(float)
        else:
            self.coefs = np.array([], dtype=float)

        # backward pruning based on GCV (try removing each basis if improves GCV)
        improved = True
        while improved and len(self.basis) > 0:
            improved = False
            # build design once per iteration
            D_full = np.ones((n, 1))
            for (a, knot, sign) in self.basis:
                proj = self._project(X, a)
                col = (np.maximum(proj - knot, 0.0) if sign == 1 else np.maximum(knot - proj, 0.0))[:, None]
                D_full = np.hstack([D_full, col])
            coef_full, *_ = np.linalg.lstsq(D_full, y, rcond=None)
            gcv_full = self._gcv(y, D_full.dot(coef_full), D_full.shape[1])
            # test removals
            for idx in range(len(self.basis)):
                basis_try = self.basis[:idx] + self.basis[idx + 1:]
                D_try = np.ones((n, 1))
                for (a, knot, sign) in basis_try:
                    proj = self._project(X, a)
                    col = (np.maximum(proj - knot, 0.0) if sign == 1 else np.maximum(knot - proj, 0.0))[:, None]
                    D_try = np.hstack([D_try, col])
                coef_try, *_ = np.linalg.lstsq(D_try, y, rcond=None)
                gcv_try = self._gcv(y, D_try.dot(coef_try), D_try.shape[1])
                if gcv_try < gcv_full - 1e-12:
                    # accept pruning
                    self.basis = basis_try
                    self.intercept_ = float(coef_try[0])
                    self.coefs = (coef_try[1:].astype(float) if D_try.shape[1] > 1 else np.array([], dtype=float))
                    improved = True
                    break
        return self

    def predict(self, X):
        X = np.asarray(X)
        n = X.shape[0]
        y_pred = np.full(n, self.intercept_, dtype=float)
        for idx, (a, knot, sign) in enumerate(self.basis):
            proj = self._project(X, a)
            term = (np.maximum(proj - knot, 0.0) if sign == 1 else np.maximum(knot - proj, 0.0))
            coef = self.coefs[idx] if idx < len(self.coefs) else 0.0
            y_pred += coef * term
        return y_pred


# ---------------------------
# LSM + faza II (L_hat, U_hat) używając ExtendedMARS
# ---------------------------
class LSM_MARS_QMC:
    def __init__(self, simulator_func, payoff_func, times, r, dim, mars_params=None, rng_seed=None):
        """
        simulator_func: callable(normals) -> paths shape (n_paths, n_steps, dim)
          where normals shape is (n_paths, n_steps, dim) standard normals.
        payoff_func: callable(t_index, x_vector) -> scalar payoff
        times: array of time grid (length n_steps)
        r: interest rate (annual)
        dim: dimension of underlying (liczba aktywów)
        mars_params: dict for ExtendedMARS
        rng_seed: optional seed for reproducibility
        """
        self.simulator = simulator_func
        self.payoff = payoff_func
        self.times = np.asarray(times)
        self.dt = np.diff(self.times)
        self.r = float(r)
        self.dim = int(dim)
        self.mars_params = mars_params or {}
        self.rng = np.random.default_rng(rng_seed)
        # model parameters (mu, sigma) needed for simulator_one_step in phase II
        self.mu = None
        self.sigma = None

    def set_model_params(self, mu, sigma):
        self.mu = np.asarray(mu, dtype=float)
        self.sigma = np.asarray(sigma, dtype=float)

    def simulator_one_step(self, x_prev, z_stdnormal, step_index):
        """
        Jednokrokowa symulacja GBM (wektorowa). Zakładamy model GBM dla każdego aktywa niezależnie.
        x_prev: (dim,)
        z_stdnormal: (dim,) standard normals
        step_index: indeks kroku docelowego (j), symulujemy z t_{j-1} -> t_j
        """
        if self.mu is None or self.sigma is None:
            raise RuntimeError("set_model_params(mu, sigma) must be called before simulator_one_step.")
        # dt for step from t_{j-1} to t_j
        if step_index <= 0:
            raise ValueError("step_index must be >= 1 for one-step simulation")
        dt = self.times[step_index] - self.times[step_index - 1]
        drift = (self.mu - 0.5 * (self.sigma ** 2)) * dt
        diffusion = self.sigma * np.sqrt(dt) * z_stdnormal
        return x_prev * np.exp(drift + diffusion)

    def _generate_normals(self, n_paths, seed_offset=0):
        n_steps = len(self.times)
        total_dim = n_steps * self.dim
        # seed_offset passed into seed for QMCPy for reproducibility; QMCPy expects integer seed or None
        seed = None if seed_offset is None else int(seed_offset)
        u = qmc_normals_qmcpy(n_paths, total_dim, seed=seed)
        normals = u.reshape(n_paths, n_steps, self.dim)
        return normals

    def price(self, N1=1024, N2=512, max_terms=12, inner_MC=64):
        """
        Wykonaj fazę I (N1) oraz fazę II (N2). Zwróć L_hat i U_hat.
        inner_MC: liczba symulacji wewnętrznych do estymacji E[h_t | X_{t-1}]
        """
        n_steps = len(self.times)
        # Phase I: fit models on N1 QMC paths
        normals1 = self._generate_normals(N1, seed_offset=11)
        X1 = self.simulator(normals1)  # (N1, n_steps, dim)
        # compute immediate payoffs matrix
        B = np.zeros((N1, n_steps))
        for i in range(N1):
            for j in range(n_steps):
                B[i, j] = float(self.payoff(j, X1[i, j, :]))

        models = [None] * n_steps
        # terminal
        models[-1] = None  # terminal value is payoff

        # backward induction LSM + MARS (fit B_j)
        for j in range(n_steps - 2, -1, -1):
            disc = np.exp(-self.r * self.dt[j])
            # target: value = max(immediate payoff, discounted next)
            Y = np.maximum(B[:, j], disc * B[:, j + 1])
            X_train = X1[:, j, :]
            model = ExtendedMARS(max_terms=max_terms, **self.mars_params)
            model.fit(X_train, Y)
            models[j] = model
            cont = model.predict(X_train)
            exercise = B[:, j] > cont
            B[:, j] = np.where(exercise, B[:, j], disc * B[:, j + 1])

        # Phase II: estimate L_hat and U_hat on independent N2 QMC paths
        normals2 = self._generate_normals(N2, seed_offset=999)
        X2 = self.simulator(normals2)
        L_vals = np.zeros(N2)
        U_vals = np.zeros(N2)

        for i in range(N2):
            pi = 0.0
            stopped = False
            for j in range(n_steps):
                t = self.times[j]
                g = float(self.payoff(j, X2[i, j, :]))
                h = float(models[j].predict(X2[i, j, :].reshape(1, -1))[0]) if models[j] else g

                if j > 0:
                    x_prev = X2[i, j - 1, :]
                    zs = self.rng.standard_normal(size=(inner_MC, self.dim))
                    x_next_samples = np.array([self.simulator_one_step(x_prev, zs[k], j) for k in range(inner_MC)])
                    h_vals = models[j].predict(x_next_samples) if models[j] else np.array([float(self.payoff(j, xn)) for xn in x_next_samples])
                    h_cond = float(np.mean(h_vals))
                else:
                    h_cond = h  # zamiast 0

                pi += h - h_cond  # bez dodatkowego dyskontowania

                # Lower bound
                if not stopped and g > h:
                    L_vals[i] = np.exp(-self.r * t) * g
                    stopped = True

                # Upper bound
                cand = np.exp(-self.r * t) * (g - pi)
                if cand > U_vals[i]:
                    U_vals[i] = cand

            if not stopped:
                L_vals[i] = np.exp(-self.r * self.times[-1]) * float(self.payoff(n_steps - 1, X2[i, -1, :]))

        L_hat = float(np.mean(L_vals))
        U_hat = float(np.mean(U_vals))
        return {"L_hat": L_hat, "U_hat": U_hat, "models": models}


# ---------------------------
# GBM simulator (użyty jako przykład)
# ---------------------------
def gbm_simulator_factory(S0, mu, sigma, times):
    """
    simulator(normals) -> paths with shape (n_paths, n_steps, dim)
    normals expected shape: (n_paths, n_steps, dim) standard normals
    """
    S0 = np.array(S0, dtype=float)
    mu = np.array(mu, dtype=float)
    sigma = np.array(sigma, dtype=float)
    times = np.array(times, dtype=float)
    dt = np.diff(times)
    n_steps = len(times)
    dim = len(S0)

    def simulator(normals):
        normals = np.asarray(normals)
        n_paths = normals.shape[0]
        paths = np.empty((n_paths, n_steps, dim), dtype=float)
        for i in range(n_paths):
            S = S0.copy()
            paths[i, 0, :] = S
            for j in range(1, n_steps):
                z = normals[i, j, :]
                S = S * np.exp((mu - 0.5 * sigma ** 2) * dt[j - 1] + sigma * np.sqrt(dt[j - 1]) * z)
                paths[i, j, :] = S
        return paths

    return simulator


# ---------------------------
# Przykładowe uruchomienie
# ---------------------------
if __name__ == "__main__":
    # parametry modelu
    S0 = [100.0]                # single asset
    mu = np.array([0.03])
    sigma = np.array([0.2])
    r = 0.03
    T = 1.0
    Nsteps = 100
    times = np.linspace(0.0, T, Nsteps + 1)

    # payoff: american put on single asset
    K = 100.0
    payoff = lambda j, x: max(K - float(np.sum(x)), 0.0)

    # stwórz symulator i pricer
    sim = gbm_simulator_factory(S0, mu, sigma, times)
    pricer = LSM_MARS_QMC(simulator_func=sim, payoff_func=payoff, times=times, r=r, dim=1,
                          mars_params={"min_leaf": 5, "penalty": 3.0}, rng_seed=12345)
    # ustaw parametry modelu potrzebne do simulator_one_step
    pricer.set_model_params(mu=mu, sigma=sigma)

    # uruchom (dobierz N1,N2,inner_MC w zależności od mocy obliczeniowej)
    for N1 in (1024, 2048, 4096, 8192):
        for N2 in (1024, 2048, 4096, 8192):
            res = pricer.price(N1=N1, N2=N2, max_terms=8, inner_MC=32)
            print(f"Przedział wyceny: L̂ = {res['L_hat']:.6f}, Û = {res['U_hat']:.6f}, N1 = {N1}, N2 = {N2}")


In [ ]:
#!/usr/bin/env python3
"""
Finalny skrypt: wycena opcji amerykańskiej (Ehrlichman-Henderson style)
- QMC (qmcpy Sobol/Halton)
- Extended MARS z LDA i GCV
- LSM + martingale control variate
- Poprawne dyskontowanie (używa dt)
"""
import numpy as np
from scipy.stats import norm
from qmcpy import Sobol, Halton
from typing import Callable, Tuple, Optional
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import warnings
warnings.filterwarnings("ignore")


# ---------------------------
# Pomocnicze funkcje
# ---------------------------

def _next_pow2(x: int) -> int:
    if x <= 0:
        return 1
    return 1 << int(np.ceil(np.log2(x)))


# ---------------------------
# Symulacja ścieżek QMC (z prefetch i dopełnieniem)
# ---------------------------

def simulate_paths(
    S0: np.ndarray,
    r: float,
    sigma: np.ndarray,
    rho: np.ndarray,
    T: int,
    N: int,
    qmc_sampler,
    seed: Optional[int] = None
) -> np.ndarray:
    """
    Generuje N ścieżek d-wymiarowego Black-Scholesa z korelacjami.
    Zwraca tablicę (N, T+1, d).
    Uwaga: zakłada czas życia = 1 (rok). dt = 1/T.
    """
    d = len(S0)
    dt = 1.0 / T
    paths = np.zeros((N, T + 1, d))
    paths[:, 0, :] = S0

    # Cholesky korelacji
    L = np.linalg.cholesky(rho)

    total_needed = N * T
    n_to_request = _next_pow2(total_needed)
    # Pobierz jednorazowo próbki QMC
    u_all = qmc_sampler.gen_samples(n_min=0, n_max=n_to_request)  # (n_to_request, d)
    u_all = u_all[:total_needed, :]

    # Zapobiegamy dokładnym 0/1 (norm.ppf -> inf)
    eps = 1e-12
    u_all = np.clip(u_all, eps, 1 - eps)

    # reshape to (T, N, d)
    try:
        u_by_t = u_all.reshape(T, N, d)
    except Exception as e:
        # awaryjnie: jeśli reshape nie działa (np. przy dziwnych wymiarach), użyj fallbacku
        u_by_t = np.zeros((T, N, d))
        for t in range(T):
            start = t * N
            u_by_t[t] = u_all[start:start + N]

    for t in range(1, T + 1):
        u = u_by_t[t - 1]  # (N, d)
        z = norm.ppf(u)    # (N, d)
        correlated_z = z @ L.T
        drift = (r - 0.5 * sigma ** 2) * dt
        diffusion = sigma * np.sqrt(dt) * correlated_z
        paths[:, t, :] = paths[:, t - 1, :] * np.exp(drift + diffusion)

    return paths


# ---------------------------
# Extended MARS z LDA + GCV
# ---------------------------

class ExtendedMARS:
    """
    Extended MARS (ridge functions): kierunki = osie + LDA; selekcja liczby terminów przez GCV.
    Model: f(x) = a0 + sum_j alpha_j * max(a_j^T x - k_j, 0)
    """
    def __init__(self, max_terms: int = 20, knot_percentiles: Optional[list] = None):
        self.max_terms = max_terms
        self.knot_percentiles = knot_percentiles or [10,20,30,40,50,60,70,80,90]
        self.a0 = 0.0
        self.alphas = []
        self.directions = []
        self.knots = []

    def _gcv(self, y, yhat, num_params):
        n = len(y)
        rss = np.sum((y - yhat)**2)
        p = max(1, num_params) + 1  # effective params (at least intercept)
        denom = (1 - p / n)**2
        if denom <= 0:
            return np.inf
        return (rss / n) / denom

    def fit(self, X: np.ndarray, y: np.ndarray):
        n, d = X.shape
        self.a0 = float(np.mean(y))
        residual = y - self.a0

        # kandydackie kierunki: osie
        directions = [np.eye(d)[i] for i in range(d)]

        # dodaj kierunek LDA (o ile możliwe)
        try:
            # utworzymy binary label przez medianę (prosty split)
            labels = (y > np.median(y)).astype(int)
            if len(np.unique(labels)) > 1:
                lda = LinearDiscriminantAnalysis(n_components=min(d, 1))
                lda.fit(X, labels)
                # scalings_ może mieć shape (d, n_components)
                scal = np.atleast_2d(lda.scalings_).T
                for vec in scal:
                    v = vec.copy().astype(float)
                    normv = np.linalg.norm(v)
                    if normv > 0:
                        v /= normv
                        directions.append(v)
        except Exception:
            # jeśli LDA się nie uda — ignorujemy
            pass

        alphas = []
        dirs = []
        knots = []

        preds = [np.full_like(y, self.a0)]
        gcv_scores = [self._gcv(y, preds[0], 1)]

        for term in range(1, self.max_terms + 1):
            best_loss = np.sum(residual**2)
            best_triplet = None
            best_h = None

            for a in directions:
                proj = X @ a
                candidates = np.percentile(proj, self.knot_percentiles)
                for k in candidates:
                    h = np.maximum(proj - k, 0.0)
                    if np.var(h) < 1e-12:
                        continue
                    alpha = float(np.dot(residual, h) / (np.dot(h,h) + 1e-12))
                    new_res = residual - alpha * h
                    loss = np.sum(new_res**2)
                    if loss < best_loss:
                        best_loss = loss
                        best_triplet = (alpha, a.copy(), float(k))
                        best_h = h.copy()

            if best_triplet is None:
                break

            alpha_star, a_star, k_star = best_triplet
            alphas.append(alpha_star)
            dirs.append(a_star)
            knots.append(k_star)
            residual -= alpha_star * best_h

            # zbuduj predykcję przy bieżącym zestawie terminów
            # unikamy sumowania pustej listy
            comp = np.zeros_like(y)
            for a_i, dir_i, k_i in zip(alphas, dirs, knots):
                comp += a_i * np.maximum(X @ dir_i - k_i, 0.0)
            pred = self.a0 + comp
            preds.append(pred)
            gcv_scores.append(self._gcv(y, pred, len(alphas)))

            # wczesne zatrzymanie, jeśli GCV się pogorszy (mały heurystyczny warunek)
            if len(gcv_scores) > 3 and gcv_scores[-1] > min(gcv_scores[:-1]):
                break

        # wybierz najlepszą liczbę terminów wg GCV
        best_idx = int(np.argmin(gcv_scores))  # index w preds (0..len(preds)-1)
        if best_idx == 0:
            # tylko stała
            self.alphas = []
            self.directions = []
            self.knots = []
        else:
            # zachowaj pierwsze best_idx terminów
            self.alphas = alphas[:best_idx]
            self.directions = dirs[:best_idx]
            self.knots = knots[:best_idx]

    def predict(self, X: np.ndarray) -> np.ndarray:
        n = X.shape[0]
        out = np.full(n, self.a0)
        for alpha, a, k in zip(self.alphas, self.directions, self.knots):
            proj = X @ a
            out += alpha * np.maximum(proj - k, 0.0)
        return out

    def conditional_expectation(self, X_prev: np.ndarray, r: float, sigma: np.ndarray, rho: np.ndarray, dt: float) -> np.ndarray:
        """
        E[h(X_t) | X_{t-1}=x_prev] dla termu h(x) = (a^T X_t - k)+ przy Black-Scholesie.
        Zakładamy, że log S_t ~ N(log S_{t-1} + (r-0.5*sigma^2) dt, cov_matrix).
        Dla h(x) = (a^T x - k)+ używamy, że a^T X_t jest kombinacją liniową wygenerowaną z lognormalnej zmiennej,
        wykonujemy przybliżenie analogiczne do Black-Scholesa na projekcji.
        """
        n, d = X_prev.shape
        res = np.zeros(n)
        cov = np.outer(sigma, sigma) * rho * dt
        for i in range(n):
            xprev = X_prev[i]
            mu_vec = np.log(xprev) + (r - 0.5 * sigma**2) * dt
            for alpha, a, k in zip(self.alphas, self.directions, self.knots):
                mean_proj = float(a @ mu_vec)
                var_proj = float(a @ cov @ a)
                if k <= 0:
                    # jeśli k <= 0 to (a^T X_t - k)+ = a^T X_t - k (zawsze dodatnie)
                    if var_proj <= 0:
                        exp_axt = np.exp(mean_proj)
                    else:
                        exp_axt = np.exp(mean_proj + 0.5 * var_proj)
                    h_exp = exp_axt - k
                else:
                    if var_proj <= 1e-16:
                        # niemal deterministyczne: przyjacielskie przybliżenie
                        mean_val = np.exp(mean_proj)
                        h_exp = max(mean_val - k, 0.0)
                    else:
                        std = np.sqrt(var_proj)
                        # u = ln(k)
                        u = np.log(k)
                        d1 = (mean_proj - u + 0.5 * var_proj) / std
                        d2 = d1 - std
                        term1 = np.exp(mean_proj + 0.5 * var_proj) * norm.cdf(d1)
                        term2 = k * norm.cdf(d2)
                        h_exp = max(term1 - term2, 0.0)
                res[i] += alpha * h_exp
        return res + self.a0


# ---------------------------
# Główna funkcja wyceny
# ---------------------------

def price_american_option(
    payoff_fn: Callable[[np.ndarray], float],
    d: int,
    n_steps: int,
    r: float,
    sigma: float,
    S0: float,
    K: float,
    N1: int = 2048,
    N2: int = 4096,
    qmc_method: str = 'sobol_scrambled',
    seed: Optional[int] = None,
    max_mars_terms: int = 15,
    rho_corr: Optional[np.ndarray] = None
) -> Tuple[float, float, float]:
    """
    Zwraca (lower_bound, upper_bound, point_estimate)
    Czas życia opcji = 1 rok; n_steps = liczba kroków (T).
    """
    T = n_steps
    dt = 1.0 / T

    if rho_corr is None:
        rho_corr = np.eye(d)
    else:
        assert rho_corr.shape == (d, d)

    S0_vec = np.full(d, S0)
    sigma_vec = np.full(d, sigma)

    # inicjalizacja QMC samplera
    if qmc_method == 'halton':
        qmc1 = Halton(dimension=d, seed=seed)
    elif qmc_method == 'sobol':
        qmc1 = Sobol(dimension=d, seed=seed)
    elif qmc_method == 'sobol_scrambled':
        qmc1 = Sobol(dimension=d, randomize='Owen', seed=seed)
    else:
        raise ValueError("Nieznana metoda QMC")

    # Faza I: dopasowanie strategii i MARS
    paths1 = simulate_paths(S0_vec, r, sigma_vec, rho_corr, T, N1, qmc1, seed)
    Y = np.array([payoff_fn(paths1[n, T, :]) for n in range(N1)])
    cv = np.zeros(N1)

    mars_models = [None] * (T + 1)
    # terminal model: tylko stała, odpowiada wartości Y
    mars_models[T] = ExtendedMARS(max_terms=1)
    mars_models[T].a0 = float(np.mean(Y))
    mars_models[T].alphas = []
    mars_models[T].directions = []
    mars_models[T].knots = []

    alpha_ls = [None] * T  # regresje LSM

    for t in range(T - 1, -1, -1):
        X_next = paths1[:, t + 1, :]
        mars = ExtendedMARS(max_terms=max_mars_terms)
        mars.fit(X_next, Y)
        mars_models[t + 1] = mars

        X_curr = paths1[:, t, :]
        exp_mars = mars.conditional_expectation(X_curr, r, sigma_vec, rho_corr, dt)
        cv += mars.predict(X_next) - exp_mars

        # ITM mask
        itm = np.array([payoff_fn(paths1[n, t, :]) > 0 for n in range(N1)])
        if not np.any(itm):
            alpha_ls[t] = np.zeros(1 + 2 * d)
            # discount Y and cv for next step
            Y *= np.exp(-r * dt)
            cv *= np.exp(-r * dt)
            continue

        X_itm = X_curr[itm]
        # basis: 1, x_i, x_i^2 (raw)
        phi = np.hstack([np.ones((X_itm.shape[0], 1)), X_itm, X_itm ** 2])
        Y_reg = Y[itm] - cv[itm]

        # least squares — stabilność
        try:
            alpha_t, *_ = np.linalg.lstsq(phi, Y_reg, rcond=None)
            alpha_t = alpha_t.ravel()
        except np.linalg.LinAlgError:
            alpha_t = np.zeros(phi.shape[1])

        alpha_ls[t] = alpha_t

        # aktualizacja decyzji (backward induction) — zastosuj dyskontowanie przy przejściu wstecz
        for n in range(N1):
            exercise = payoff_fn(paths1[n, t, :])
            if itm[n]:
                cont = float(alpha_t @ np.hstack([1.0, paths1[n, t, :], paths1[n, t, :] ** 2]))
            else:
                cont = -np.inf
            if exercise > cont:
                Y[n] = exercise
                cv[n] = 0.0
            else:
                # discount to next time step
                Y[n] *= np.exp(-r * dt)
                cv[n] *= np.exp(-r * dt)

    # Faza II: estymacja ograniczeń
    # drugi sampler QMC (inna sekwencja przez zmieniony seed)
    if qmc_method == 'halton':
        qmc2 = Halton(dimension=d, seed=(seed + 1) if seed is not None else None)
    elif qmc_method == 'sobol':
        qmc2 = Sobol(dimension=d, seed=(seed + 1) if seed is not None else None)
    elif qmc_method == 'sobol_scrambled':
        qmc2 = Sobol(dimension=d, randomize='Owen', seed=(seed + 1) if seed is not None else None)
    else:
        raise ValueError("Nieznana metoda QMC")

    paths2 = simulate_paths(S0_vec, r, sigma_vec, rho_corr, T, N2, qmc2, seed)

    # Martyngał kontrolny pi: akumulacja z dyskontowaniem zgodnie z formułą
    # pi_t = pi_{t-1} + exp(-r*(t-1)*dt) * (h_t - E[h_t|F_{t-1}])
    pi = np.zeros((N2, T + 1))
    for t in range(1, T + 1):
        X_t = paths2[:, t, :]
        h_t = mars_models[t].predict(X_t)
        X_prev = paths2[:, t - 1, :]
        exp_h = mars_models[t].conditional_expectation(X_prev, r, sigma_vec, rho_corr, dt)
        pi[:, t] = pi[:, t - 1] + np.exp(-r * (t - 1) * dt) * (h_t - exp_h)

    # Strategia wykonania (LSM używając alpha_ls)
    tau = np.full(N2, T, dtype=int)
    for n in range(N2):
        for t in range(T):
            exercise = payoff_fn(paths2[n, t, :])
            alpha_t = alpha_ls[t]
            if alpha_t is None:
                cont = -np.inf
            else:
                cont = float(alpha_t @ np.hstack([1.0, paths2[n, t, :], paths2[n, t, :] ** 2]))
            if exercise > cont:
                tau[n] = t
                break

    # zdyskontowane wypłaty wykonania
    disc_payoff = np.exp(-r * tau * dt) * np.array([payoff_fn(paths2[n, tau[n], :]) for n in range(N2)])
    lower = np.mean(disc_payoff - pi[np.arange(N2), tau])

    # upper bound: sup_t [discounted payoff - pi_t]
    upper_vals = []
    for n in range(N2):
        vals = [np.exp(-r * t * dt) * payoff_fn(paths2[n, t, :]) - pi[n, t] for t in range(T + 1)]
        upper_vals.append(max(vals))
    upper = np.mean(upper_vals)

    # punktowa estymata: średnia LB/UB (możesz też użyć lower)
    point = 0.5 * (lower + upper)

    return lower, upper, point


# ---------------------------
# Przykład użycia (test)
# ---------------------------

if __name__ == "__main__":
    # parametry testowe (1-roczna opcja, T kroków)
    d = 1
    T = 30         # liczba kroków (miesięczna dyskretyzacja)
    r = 0.05
    sigma = 0.2
    S0 = 100.0
    K = 100.0
    rho = np.eye(d) * 1.0
    payoff = lambda x: max(x[0] - K, 0.0)  # single-asset call (dla d=1)

    lb, ub, pe = price_american_option(
        payoff_fn=payoff,
        d=d,
        n_steps=T,
        r=r,
        sigma=sigma,
        S0=S0,
        K=K,
        N1=1024,
        N2=1024,
        qmc_method='sobol_scrambled',
        seed=42,
        max_mars_terms=12,
        rho_corr=rho
    )

    print(f"Dolne ograniczenie: {lb:.4f}")
    print(f"Górne ograniczenie: {ub:.4f}")
    print(f"Szacowana cena:     {pe:.4f}")
    print(f"Szerokość przedziału: {ub - lb:.4f}")
